# Ant Colony Optimization

# Documentation

## tsp module

`cost(solution, distance_matrix, round_trip)`: returns the cost of a solution.
* *solution (np.ndarray):* list of cities indexes in the order they are visited.
* *distance_matrix (np.ndarray):* distance matrix between all cities.
* *round_trip (bool):* if True, the solution is a round trip. Default: True.

`read_tsp_file(file_name)`: reads data from a file and returns a pandas DataFrame.
* *file_name (str):* file name to read


`distance(city1, city2)`: returns the distance between two cities.
* *city1 (tuple[int, int]):* first city coordinates.
* *city2 (tuple[int, int]):* second city coordinates.
  
`distance_matrix(df)`: returns the distance matrix between all cities.
* *df (pandas.DataFrame):* DataFrame containing the cities coordinates.

`plot_solution_path(df, solution, figsize, path_alpha)`: plots the solution path.
* *df (pandas.DataFrame):* DataFrame containing the cities coordinates.
* *solution (list[int]):* list of cities indexes in the order they are visited.
* *figsize (tuple[int, int]):* figure size. Default: (10, 10).
* *path_alpha (float):* path transparency. Default: 0.5.


## ant_system module

`AntSystem(distance_matrix, alpha, beta, evaporation_rate, n_ants, round_trip, rng)`: Ant System implementation.
* *distance_matrix (np.ndarray):* distance matrix between all cities.
* *alpha (float):* pheromone influence ($\alpha \geq 0$).
* *beta (float):* distance influence ($\beta \geq 0$).
* *evaporation_rate (float):* pheromone evaporation rate ($0 \leq \rho \leq 1$).
* *n_ants (int):* number of ants ($n \geq 1$).
* *round_trip (bool):* if True, the solution is a round trip.
* *rng (np.random.Generator):* random number generator.


`AntSystem.initialization()`: initializes the tabu list. Each ant starts from a random city.

`AntSystem.next_city(ant, current_city_index)`: returns the next city to visit.
* *ant (int):* ant number.
* *current_city_index (int):* current city index of the tabu list.
* *Returns (int):* next city index.

`AntSystem.cycle()`: runs a cycle of the algorithm.

`AntSystem.cycle_best_solutions(n)`: returns the $n$ best solutions of the cycle.
* *n (int):* number of best solutions to return.
* *Returns (np.ndarray):* n array of shape (n, n_cities) containing the $n$ best solutions.


`AntSystem.evaporation()`: evaporates the pheromones.

`AntSystem.reinforcement()`: reinforce the pheromones.

`AntSystem.run(max_cycles, verbose)`: runs the algorithm.
* *max_cycles (int):* maximum number of cycles.
* *verbose (bool):* if True, prints the best solution and its cost at each cycle. Default: False.
* *Returns (np.ndarray):* best solution found.
